<a href="https://colab.research.google.com/github/Alpereninci1/ProjectOfGraduation/blob/main/bitirmeProjesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/opencv/opencv
!git clone https://github.com/opencv/opencv_contrib

Cloning into 'opencv'...
remote: Enumerating objects: 302995, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 302995 (delta 6), reused 21 (delta 1), pack-reused 302962
Receiving objects: 100% (302995/302995), 492.09 MiB | 25.88 MiB/s, done.
Resolving deltas: 100% (210826/210826), done.
Checking out files: 100% (6992/6992), done.
Cloning into 'opencv_contrib'...
remote: Enumerating objects: 36527, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 36527 (delta 65), reused 107 (delta 38), pack-reused 36350
Receiving objects: 100% (36527/36527), 132.24 MiB | 29.29 MiB/s, done.
Resolving deltas: 100% (22635/22635), done.


In [5]:
!mkdir build
%cd build

/content/build


In [6]:
!cmake -D OPENCV_EXTRA_MODULES_PATH=../opencv_contrib/modules -DBUILD_SHARED_LIBS=OFF -DBUILD_TESTS=OFF -D BUILD_PERF_TESTS=OFF -D BUILD_EXAMPLES=OFF -D WITH_OPENEXR=OFF -D WITH_CUDA=ON -D WITH_CUBLAS=ON -D WITH_CUDNN=ON -D OPENCV_DNN_CUDA=ON ../opencv

-- 'Release' build type is used by default. Use CMAKE_BUILD_TYPE to specify build type (Release or Debug)
-- The CXX compiler identification is GNU 7.5.0
-- The C compiler identification is GNU 7.5.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Detected processor: x86_64
-- Found PythonInterp: /usr/bin/python2.7 (found suitable version "2.7.17", minimum required is "2.7") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython2.7.so (found suitable exact version "2.7.17") 
-- Found PythonInterp: /usr/bin/python3 (found suitable versio

In [ ]:
!make -j8 install

Scanning dependencies of target opencv_highgui_plugins
Scanning dependencies of target ippiw
Scanning dependencies of target ittnotify
Scanning dependencies of target quirc
Scanning dependencies of target ade
Scanning dependencies of target gen_opencv_python_source
Scanning dependencies of target libprotobuf
Scanning dependencies of target opencv_videoio_plugins
[  0%] Built target opencv_highgui_plugins
[  0%] Building C object 3rdparty/quirc/CMakeFiles/quirc.dir/src/decode.c.o
[  0%] Built target opencv_videoio_plugins
[  0%] Building C object 3rdparty/quirc/CMakeFiles/quirc.dir/src/quirc.c.o
[  0%] Building C object 3rdparty/ittnotify/CMakeFiles/ittnotify.dir/src/ittnotify/ittnotify_static.c.o
[  0%] Building C object 3rdparty/quirc/CMakeFiles/quirc.dir/src/version_db.c.o
[  0%] Generate files for Python bindings and documentation
[  0%] Building CXX object CMakeFiles/ade.dir/3rdparty/ade/ade-0.1.1f/sources/ade/source/alloc.cpp.o
[  0%] Building C object 3rdparty/ippiw/CMakeFiles/ip

In [1]:
import cv2
import time
cv2.__version__

In [3]:
CONFIDENCE_THRESHOLD = 0.2
NMS_THRESHOLD = 0.4
COLORS = [(0, 255, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]

class_names = []
with open('obj.names', 'r') as f:
    class_names = [cname.strip() for cname in f.readlines()]

#vc = cv2.VideoCapture("demo.mp4") #video
vc=cv2.VideoCapture(0)#webcam
net = cv2.dnn.readNet("yolov4-obj_last.weights", "yolov4-obj.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)

while cv2.waitKey(1) < 1:
    (grabbed, frame) = vc.read()
    if not grabbed:
        exit()

    start = time.time()
    classes, scores, boxes = model.detect(frame, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    end = time.time()

    start_drawing = time.time()
    for (classid, score, box) in zip(classes, scores, boxes):
        color = COLORS[int(classid) % len(COLORS)]
        label = "%s : %f" % (class_names[classid], score)
        cv2.rectangle(frame, box, color, 2)
        cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    end_drawing = time.time()

    fps_label = "FPS: %.2f (excluding drawing time of %.2fms)" % (1 / (end - start), (end_drawing - start_drawing) * 1000)
    cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.imshow("detections", frame)

error: ignored